In [1]:
%matplotlib inline
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import lightgbm as lgb
import matplotlib.pylab as plt
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train = pd.read_csv('jinnan_round1_train_20181227.csv',encoding='gbk')
test = pd.read_csv('jinnan_round1_testA_20181227.csv',encoding='gbk')

In [3]:
for df in [train, test]:
    df.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)

In [4]:
# 删除缺失率超过90%的列
good_cols = list(train.columns)
for col in train.columns:
    rate = train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        good_cols.remove(col)

# 删除异常值
train = train[train['收率']>0.87]
        
train = train[good_cols]
good_cols.remove('收率')
test  = test[good_cols]

In [5]:
target = train['收率']
del train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

In [6]:
def timeTranSecond(t):
    try:
        t,m,s=t.split(":")
    except:
        if t=='1900/1/9 7:00':
            return 7*3600/3600
        elif t=='1900/1/1 2:30':
            return (2*3600+30*60)/3600
        elif t==-1:
            return -1
        else:
            return 0
    try:
        tm = (int(t)*3600+int(m)*60+int(s))/3600
    except:
        return (30*60)/3600
    
    return tm
for f in ['A5','A7','A9','A11','A14','A16','A24','A26','B5','B7']:
    data[f] = data[f].apply(timeTranSecond)

def getDuration(se):
    try:
        sh,sm,eh,em=re.findall(r"\d+\.?\d*",se)
    except:
        if se == -1:
            return -1 
        
    try:
        if int(sh)>int(eh):
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600 + 24
        else:
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600
    except:
        if se=='19:-20:05':
            return 1
        elif se=='15:00-1600':
            return 1
    
    return tm
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f] = data.apply(lambda df: getDuration(df[f]), axis=1)

In [7]:
cate_columns = [f for f in data.columns if f != '样本id']

In [8]:
for f in cate_columns:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
train = data[:train.shape[0]]
test  = data[train.shape[0]:]

In [9]:
train['target'] = target
train['intTarget'] = pd.cut(train['target'], 5, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
li = ['intTarget_0.0','intTarget_1.0','intTarget_2.0','intTarget_3.0','intTarget_4.0']
mean_features = []

for f1 in cate_columns:
    for f2 in li:
        col_name = f1+"_"+f2+'_mean'
        mean_features.append(col_name)
        order_label = train.groupby([f1])[f2].mean()
        for df in [train, test]:
            df[col_name] = df[f].map(order_label) # 正是此处逻辑错误

train.drop(li, axis=1, inplace=True)

In [10]:
train_2 = train.drop(['样本id','target'], axis=1)
test = test[train_2.columns]
X_train = train_2.values
Y_train = target.values
X_test = test.values

In [11]:
Y_train.shape

(1381,)

In [12]:
X_train.shape

(1381, 192)

In [13]:
Z_data = pd.DataFrame(X_train)
Z_data = Z_data.fillna('0')
#Z_data = Z_data.convert_objects(convert_numeric=True)
Z_data.shape

(1381, 192)

In [14]:
x_train,x_test,y_train,y_test = train_test_split(Z_data,Y_train,train_size = 0.8,random_state=100)

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation ,Dropout
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(100,input_dim=192))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dense(1))

          
sgd = SGD(lr=0.0005)
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['mse'])

for step in range(2001):
    cost = model.train_on_batch(x_train, y_train)
    if step % 400 == 0:
        print("After %d trainings, the cost: %f" % (step, cost[0]))
        print('RMSE：%f' % mean_squared_error(y_test,model.predict(x_test)))

Using TensorFlow backend.


After 0 trainings, the cost: 4.464980
RMSE：0.927552
After 400 trainings, the cost: 0.274762
RMSE：0.163742
After 800 trainings, the cost: 0.162563
RMSE：0.094828
After 1200 trainings, the cost: 0.119484
RMSE：0.067119
After 1600 trainings, the cost: 0.093837
RMSE：0.051411
After 2000 trainings, the cost: 0.072791
RMSE：0.041321


**使用CV对以下几个模型作改良**

In [17]:
kfold = cross_validation.KFold(len(Z_data), n_folds=5)

In [18]:
rf = RandomForestRegressor(n_estimators= 50, max_depth=20, min_samples_split=10,
                                  min_samples_leaf=3,max_features='sqrt' ,oob_score=True, random_state=10).fit(x_train,y_train)
print('r f：%f' % mean_squared_error(y_test,rf.predict(x_test)))

r f：0.000166


In [19]:
cross_validation.cross_val_score(rf, Z_data, Y_train, n_jobs=-1,scoring='neg_mean_squared_error')

array([-0.00020545, -0.00019516, -0.00019057])

In [20]:
bdt = AdaBoostRegressor(DecisionTreeRegressor(max_depth=30, min_samples_split=15, min_samples_leaf=10,criterion='mse'),loss='square',
                         n_estimators=20, learning_rate=0.01)
bdt.fit(x_train,y_train)
print('bdt: %.6f '%mean_squared_error(y_test,bdt.predict(x_test)))

bdt: 0.000184 


In [21]:
cross_validation.cross_val_score(bdt, Z_data, Y_train, n_jobs=-1,scoring='neg_mean_squared_error')

array([-0.00021037, -0.00021701, -0.00018989])

In [22]:
gbm = GradientBoostingRegressor(learning_rate=0.05, n_estimators=50,max_depth=20, min_samples_leaf =10, 
                                min_samples_split =8, max_features='sqrt', subsample=0.9, random_state=10)

gbm.fit(x_train,y_train)
print('gbm: %.6f' %mean_squared_error(y_test,gbm.predict(x_test)))

gbm: 0.000170


In [23]:
cross_validation.cross_val_score(gbm, Z_data, Y_train, n_jobs=-1,scoring='neg_mean_squared_error')

array([-0.00020685, -0.00019559, -0.00019638])

In [24]:
x_train,x_test,y_train,y_test = train_test_split(Z_data,Y_train,train_size = 0.5,random_state=140)

In [25]:
stack = pd.DataFrame()

stack['rf'] = rf.predict(x_test)
stack['bdt'] = bdt.predict(x_test)
stack['gbm'] = gbm.predict(x_test)
#############################################

rg = RidgeCV(cv=5).fit(stack,y_test)
coefs = rg.coef_
intercepts = rg.intercept_
print(coefs,intercepts)
print(mean_squared_error(y_test,rg.predict(stack)))

[0.31532057 0.38181265 0.30986591] -0.006505678738667253
0.00014349832290817024


In [26]:
X_test1 = pd.DataFrame(X_test).fillna(0)

In [27]:
FFF = pd.DataFrame()

In [28]:
FFF['rf'] = rf.predict(X_test1)
FFF['bdt'] = bdt.predict(X_test1)
FFF['gbm'] = gbm.predict(X_test1)

finals = rg.predict(FFF)

commit = pd.DataFrame()
commit['result'] = finals
commit = commit.apply(lambda x:round(x,3))
commit.to_csv('fff.csv',header=None,index=False)

print('file saved!')

file saved!
